In [39]:
# Importando as bibliotecas necessárias
import pandas as pd
from sklearn_crfsuite import CRF
import json
from sklearn.model_selection import train_test_split

In [40]:
# Carregar os dados do arquivo jsonl
with open('final_data.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# Converter os dados para o formato desejado
sentences = []
for item in data:
    text = item['text']
    entities = item['entities']
    words = []
    start_idx = 0
    for start, end, label in entities:
        # Adicionar palavras antes da entidade atual (se houver)
        words.extend([(word, 'O') for word in text[start_idx:start].split()])
        # Adicionar a entidade atual
        words.extend([(word, label) for word in text[start:end].split()])
        start_idx = end
    # Adicionar palavras após a última entidade
    words.extend([(word, 'O') for word in text[start_idx:].split()])
    sentences.append(words)

In [41]:
sentences

[[('introdução', 'Nao Risco'),
  ('1.1', 'Nao Risco'),
  ('.', 'O'),
  ('objetivo', 'Nao Risco'),
  ('propósito', 'Nao Risco'),
  ('fundamental', 'Nao Risco'),
  ('deste', 'Nao Risco'),
  ('documento', 'Nao Risco'),
  ('articular', 'Nao Risco'),
  ('clareza', 'Nao Risco'),
  ('minúcia', 'Nao Risco'),
  ('requisitos', 'Nao Risco'),
  ('técnicos', 'Nao Risco'),
  ('operacionais', 'Nao Risco'),
  ('sistema', 'Nao Risco'),
  ('monitoramento', 'Nao Risco'),
  ('hidrelétricas', 'Nao Risco'),
  ('smh', 'Nao Risco')],
 [('projetado', 'Nao Risco'),
  ('ferramenta', 'Nao Risco'),
  ('crucial', 'Nao Risco'),
  ('visa', 'Nao Risco'),
  ('atender', 'Nao Risco'),
  ('complexa', 'Nao Risco'),
  ('rede', 'Nao Risco'),
  ('profissionais', 'Nao Risco'),
  ('envolvidos', 'Nao Risco'),
  ('desde', 'Nao Risco'),
  ('engenheiros', 'Nao Risco'),
  ('analistas', 'Nao Risco'),
  ('responsáveis', 'Nao Risco'),
  ('tomada', 'Nao Risco'),
  ('decisões', 'Nao Risco'),
  ('organizações', 'Nao Risco'),
  ('hidrelétr

In [42]:
# Funções para extração de características e rótulos
def word2features(sent, i):
    word = sent[i][0]
    label = sent[i][1]
    features = {
        'bias': '1.0',
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': str(word.isupper()),
        'word.istitle()': str(word.istitle()),
        'word.isdigit()': str(word.isdigit()),
        'label': label
    }
    return features

In [43]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [44]:
def sent2labels(sent):
    return [label for word, label in sent]

In [45]:
# Separe os dados em treino e teste
sentences_train, sentences_test = train_test_split(sentences, test_size=0.8)

In [46]:
# Preparação dos dados para o modelo - TREINO
X_train = [sent2features(s) for s in sentences_train]
y_train = [sent2labels(s) for s in sentences_train]

In [47]:
# Preparação dos dados para o modelo - TESTE
X_test = [sent2features(s) for s in sentences_test]
y_test = [sent2labels(s) for s in sentences_test]

In [54]:
# Criação do modelo CRF
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=5,
    all_possible_transitions=True
)

In [55]:
# Treinamento do modelo
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass
# Use o modelo treinado para prever as tags nas sentenças de teste
y_pred = crf.predict(X_test)

In [58]:
from sklearn.metrics import classification_report, accuracy_score

# Flatten (achatar) os dados
y_true_flat = [label for sentence in y_test for label in sentence]
y_pred_flat = [label for sentence in y_pred for label in sentence]

# Calcular e imprimir as métricas
print("Relatório de Classificação:")
print(classification_report(y_true_flat, y_pred_flat))
print("Acurácia:", accuracy_score(y_true_flat, y_pred_flat))

Relatório de Classificação:
              precision    recall  f1-score   support

   Nao Risco       1.00      1.00      1.00      1173
           O       0.00      0.00      0.00       124
       Risco       0.69      1.00      0.81       263

    accuracy                           0.92      1560
   macro avg       0.56      0.67      0.60      1560
weighted avg       0.87      0.92      0.89      1560

Acurácia: 0.9205128205128205


C:\Users\kevin\PycharmProjects\ModeloCRF\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kevin\PycharmProjects\ModeloCRF\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kevin\PycharmProjects\ModeloCRF\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r